# ファイル名を取得

In [114]:
import os
import pandas as pd
import numpy as np
import re

In [121]:
def set_range(range_str):
    if pd.isna(range_str):  # NaNの場合
        return pd.Series([np.nan, np.nan])
    
    # 小数点を考慮した正規表現
    # 「X以上Y未満」のパターン
    match_between = re.match(r'([0-9]+(\.[0-9]+)?)以上([0-9]+(\.[0-9]+)?)未満', range_str)
    if match_between:
        lower_bound = float(match_between.group(1))
        upper_bound = float(match_between.group(3))  # Y未満なのでY
        return pd.Series([lower_bound, upper_bound])
    
    # 「X以上」のパターン
    match_above = re.match(r'([0-9]+(\.[0-9]+)?)以上', range_str)
    if match_above:
        lower_bound = float(match_above.group(1))
        return pd.Series([lower_bound, float('inf')])  # 上限は無限大
    
    # 「Y未満」のパターン
    match_below = re.match(r'([0-9]+(\.[0-9]+)?)未満', range_str)
    if match_below:
        upper_bound = float(match_below.group(1))  # Y未満なのでY
        return pd.Series([0, upper_bound])  # 下限は0
    
    # その他のケース
    return pd.Series([np.nan, np.nan])  # NaNの場合

In [122]:
# カレントディレクトリのファイル名を取得
directory_path = './'  # 作業しているディレクトリのパス
file_names = [f for f in os.listdir(directory_path) if f.endswith('.xlsx')]

file_names

['中性脂肪.xlsx',
 '血清クレアチニン.xlsx',
 'HDL-C.xlsx',
 '尿糖.xlsx',
 'γ-GTP.xlsx',
 'BMI.xlsx',
 '腹囲.xlsx',
 'HbA1C.xlsx',
 '空腹時血糖.xlsx',
 'GPT(ALT).xlsx',
 '拡張期血圧.xlsx',
 'eGFR.xlsx',
 'GOT(AST).xlsx',
 '尿蛋白.xlsx',
 '収縮期血圧.xlsx',
 'LDL-C.xlsx']

# データ読み込みと前処理

In [123]:
# ファイルごとのデータフレームを格納するリスト
dfs = []

# ファイルを順番に読み込み、データフレームとしてリストに格納
for file in file_names:
    df = pd.read_excel(file, skiprows=1)
    dfs.append(df)

In [124]:
dfs[0]
file_names[0]

'中性脂肪.xlsx'

In [125]:

# データフレームを格納するリスト
dfs = []

# 各 Excel ファイルに対して処理を行う
for file in file_names:
    # データを読み込む（最初の行をヘッダーとして使用しない）
    df = pd.read_excel(file, header=None)

    # 1行目のヘッダー情報を補正して、性別と年齢層に分ける
    df.columns = ['都道府県', '検査値範囲', '男_40～44歳', '男_45～49歳', '男_50～54歳', 
                  '男_55～59歳', '男_60～64歳', '男_65～69歳', '男_70～74歳', '男_中計',
                  '女_40～44歳', '女_45～49歳', '女_50～54歳', '女_55～59歳', 
                  '女_60～64歳', '女_65～69歳', '女_70～74歳', '女_中計']

    # 男_中計と女_中計列を削除
    df = df.drop(columns=['男_中計', '女_中計'])

    # NaNのある行は必要なデータではないので削除
    df = df.dropna(how='all')

    # 0~2行目を削除する
    df = df.drop(index=[0, 1, 2]).reset_index(drop=True)

    # '地域' 列の NaN を直前の値で埋める
    df['都道府県'] = df['都道府県'].ffill()

    # 地域が「都道府県判別不可」の行を削除
    df = df[df['都道府県'] != '都道府県判別不可']
    
    # 0~1行目を削除する
    df = df.drop(index=[0, 1]).reset_index(drop=True)

    # 男性データを整理
    df_men = df[['都道府県', '検査値範囲', '男_40～44歳', '男_45～49歳', '男_50～54歳',
                  '男_55～59歳', '男_60～64歳', '男_65～69歳', '男_70～74歳']]
    df_men = pd.melt(df_men, id_vars=['都道府県', '検査値範囲'], 
                     var_name='年齢区分', value_name='人数')
    df_men['性別'] = '男性'
    df_men['年齢区分'] = df_men['年齢区分'].str.replace('男_', '')

    # 女性データを整理
    df_women = df[['都道府県', '検査値範囲', '女_40～44歳', '女_45～49歳', '女_50～54歳',
                   '女_55～59歳', '女_60～64歳', '女_65～69歳', '女_70～74歳']]
    df_women = pd.melt(df_women, id_vars=['都道府県', '検査値範囲'], 
                       var_name='年齢区分', value_name='人数')
    df_women['性別'] = '女性'
    df_women['年齢区分'] = df_women['年齢区分'].str.replace('女_', '')

    # 男性と女性のデータを結合
    df_cleaned = pd.concat([df_men, df_women], ignore_index=True)
    
    # '下限' と '上限' の新しい列を追加
    df_cleaned[['下限値', '上限値']] = df_cleaned['検査値範囲'].apply(set_range)

    
    # 男_中計と女_中計列を削除
    df_cleaned = df_cleaned.drop(columns=['検査値範囲'])    
    
    # 列の順番を変更
    df_cleaned = df_cleaned[['都道府県', '性別', '年齢区分', '下限値', '上限値', '人数']]
    
    # CSVファイルとして保存
    csv_file_name = file.replace('.xlsx', '.csv')
    df_cleaned.to_csv("data/" + csv_file_name, index=False, encoding='utf-8-sig')

print("すべてのファイルを処理し、保存しました。")

すべてのファイルを処理し、保存しました。


In [126]:
df_cleaned

,都道府県,性別,年齢区分,下限値,上限値,人数
0,北海道,男性,40～44歳,180.0,inf,4836
1,北海道,男性,40～44歳,160.0,180.0,7958
2,北海道,男性,40～44歳,140.0,160.0,14689
3,北海道,男性,40～44歳,120.0,140.0,20193
4,北海道,男性,40～44歳,100.0,120.0,18756
...,...,...,...,...,...,...
3943,沖縄県,女性,70～74歳,160.0,180.0,1345
3944,沖縄県,女性,70～74歳,140.0,160.0,2832
3945,沖縄県,女性,70～74歳,120.0,140.0,4152
3946,沖縄県,女性,70～74歳,100.0,120.0,4083
